In [9]:
import numpy
import numpy.random as rng
from IPython.display import SVG

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import theano
import theano.tensor as T
import theano.d3printing as d3p

# Logistic regression 

In [12]:
# Training data
N = 400
feats = 784
D = (rng.randn(N, feats).astype(theano.config.floatX), rng.randint(size=N,low=0, high=2).astype(theano.config.floatX))
training_steps = 10000

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats).astype(theano.config.floatX), name="w")
b = theano.shared(numpy.asarray(0., dtype=theano.config.floatX), name="b")
x.tag.test_value = D[0]
y.tag.test_value = D[1]

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w)-b)) # Probability of having a one
prediction = p_1 > 0.5 # The prediction that is done: 0 or 1

# Compute gradients
xent = -y*T.log(p_1) - (1-y)*T.log(1-p_1) # Cross-entropy
cost = xent.mean() + 0.01*(w**2).sum() # The cost to optimize
gw,gb = T.grad(cost, [w,b])

# Training and prediction function
train = theano.function(inputs=[x,y], outputs=[prediction, xent],
                        updates=[[w, w-0.01*gw], [b, b-0.01*gb]],
                        name = "train", profile=True)
predict = theano.function(inputs=[x], outputs=prediction,
                          name = "predict", profile=True)

# Example 1 

In [13]:
d3p.d3print(predict, 'example1/index.html')

[open](./example1/index.html)

# Example 2 

In [14]:
d3p.d3print(train, 'example2/index.html')

[open](./example2/index.html)

# Example 3 (profiling)

In [15]:
n = 100
m = 50
h = 10

W = theano.shared(np.random.normal(0, 1, (m, h)).astype(theano.config.floatX), borrow=True)
X = T.dmatrix('X')
Z = T.nnet.sigmoid(T.dot(X, W))
zm = T.mean(Z)

predict_profiled = theano.function([X], [Z, zm], profile=True)
data = rng.rand(n, m)
dZ, dzm = predict_profiled(data)

In [16]:
d3p.d3print(predict_profiled, './example3/index.html')

[open](./example3/index.html)